In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold,GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from sklearn.inspection import permutation_importance


from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import time


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df=pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv',index_col='id')
x_test=pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv',index_col='id')
display(df.head())
display(x_test.head())

In [ ]:
display(df.describe().T)
print(df.info())
print(df.select_dtypes('object'))
print(df.shape)
print(x_test.shape)
df.isnull().sum()

In [ ]:
sns.catplot(x="target", kind="count", palette="ch:.25", data=df)

# plot some of columns to get intition about data distribution

In [ ]:
plt.figure(figsize=(25, 25))
for i, col in enumerate(list(df.columns)[2:30]):
    plt.subplot(7, 4, i+1)
    sns.histplot(df[col], kde=True, bins=10)

In [ ]:
# grid = sns.PairGrid(df.iloc[:,1:20])
# grid.map_offdiag(sns.scatterplot)
# grid.map_diag(sns.histplot)
# grid.add_legend()

# see percent of missing value

In [ ]:
print(df.isnull().sum())
titanic_null_count=df.isnull().sum()*100/len(df)
miss_titanic_plot=titanic_null_count.sort_values(ascending=False)[:10].plot(kind='barh')
print(titanic_null_count.sort_values(ascending=False))

In [ ]:
# correlation=abs(df.corr()['target']) >= 0.4
# correlation[correlation==True]

In [ ]:
df.shape

**data has 0 missing values**

In [ ]:
Y=df['target']
x=df.drop(['target'],axis=1)

In [ ]:
over = SMOTE()
# under = RandomUnderSampler()

In [ ]:
# steps = [('o', over), ('u', under)]
# pipeline = Pipeline(steps=steps)

In [ ]:
x, Y = over.fit_resample(x, Y)
print(x.shape)
print(Y.shape)

In [ ]:
display(x.describe().T)

In [ ]:
sns.catplot(x=Y, kind="count", palette="ch:.25", data=df)

In [ ]:
scaler=RobustScaler()
x_cols=x.columns
x_test_cols=x_test.columns
x=pd.DataFrame(scaler.fit_transform(x.astype(float)),columns=x_cols)
x.index=x.index

x_test=pd.DataFrame(scaler.fit_transform(x_test.astype(float)),columns=x_test_cols,index=x_test.index)
display(x.head())
display(x.tail())


In [ ]:
noise = np.random.normal(0, 0.1, x.shape) 
x=x+noise
print(x.shape)
display(x.head())
display(x.tail())

In [ ]:
pca = PCA().fit(x)
plt.rcParams["figure.figsize"] = (20,10)

fig, ax = plt.subplots()

y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

In [ ]:
# def reduce_data(df,comp_number=280):
#     pca = PCA(n_components=comp_number).fit(df)
#     reduced_data = pca.transform(df)
#     reduced_data = pd.DataFrame(reduced_data)
#     print(pca.explained_variance_ratio_.sum())
#     return reduced_data

In [ ]:
# x_red=reduce_data(x)
# x_test_red=reduce_data(x_test)

In [ ]:
x_red=x
x_test_red=x_test

In [ ]:
x_red.shape

In [ ]:
# x_train, x_val, y_train, y_val=train_test_split(x_red, Y, test_size=0.20,random_state=42,shuffle=True)

In [ ]:
random = RandomForestClassifier()
logreg = LogisticRegression()
svm=SVC()
gbc = GradientBoostingClassifier()
xgb=XGBClassifier()
gnb=GaussianNB()

In [ ]:
x_red=x_red[['127','176','175','66','59']]
x_test_red=x_test_red[['127','176','175','66','59']]

In [ ]:
# class_list = [random,logreg,svm,gbc,xgb,gnb]
# for model in class_list:
#         start = time.time()
#         grid = GridSearchCV(estimator=model, param_grid={}, scoring='roc_auc', cv=5, n_jobs=-1,verbose=3)
#         grid.fit(x_red, Y)
#         end = time.time()
#         print(model, '\n', grid.best_score_,'\n', round(end-start))

In [ ]:
# param_grid = {'warm_start': [True],
#               'bootstrap': [True],
#               'oob_score':[True],
#              'n_estimators':[70,80,90,100,120],
#              'max_depth':[10,15,20]}
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# best = GridSearchCV(estimator=ExtraTreesClassifier(), param_grid=param_grid, scoring='roc_auc', cv=kfold, n_jobs=-1,verbose=3)
# best.fit(x_red, Y)

# print(best.best_score_,best.best_estimator_)

In [ ]:
###### param_grid = {'C': [0.03,0.05,0.08,0.1,0.2,0.5,1,1.5,2],
#               'penalty': ['l1','l2','elasticnet'],
#              'class_weight':[None,'balanced'],
#              'solver':['lbfgs', 'liblinear','newton-cg'],
#              'max_iter':[50,100,300,500,1000]}
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# best = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, scoring='roc_auc', cv=kfold, n_jobs=-1,verbose=3)
# best.fit(x_red, Y)

# print(best.best_score_,best.best_estimator_)

In [ ]:
# param_grid = {'C': [0.01,0.05,0.1,0.2,1,2,5],
#               'kernel': [ 'poly','sigmoid', 'linear','rbf'],
#              'degree':[2,3,4],
#              'gamma':['scale','auto']}
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# best = GridSearchCV(estimator=SVC(), param_grid=param_grid, scoring='roc_auc', cv=kfold, n_jobs=-1,verbose=3)
# best.fit(x_red, Y)

# print(best.best_score_,best.best_estimator_)

In [ ]:
param_grid = {'n_estimators': [50,100,150,300,500,1000],
              'min_samples_split': [2,3,4,5,6],
             'warm_start':[True,False]}
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

best = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring='roc_auc', cv=kfold, n_jobs=-1,verbose=3)
best.fit(x_red, Y)

print(best.best_score_,best.best_estimator_,best.best_params_)

In [ ]:
final_model=RandomForestClassifier(n_estimators=150, warm_start=True)
final_model.fit(x_red,Y)

In [ ]:
features=x_red.columns
importances = final_model.feature_importances_
indices = np.argsort(importances)
# indices=indices[280:]
# print(indices)
plt.figure(figsize=(20,50))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


In [ ]:
# final_model=RandomForestClassifier(n_estimators=150, warm_start=True)
# final_model.fit(x_red,Y)
final_predict=final_model.predict(x_test_red)
submission = pd.DataFrame({'id': x_test.index,'target': final_predict})
submission

In [ ]:
submission.to_csv('/kaggle/working/grid_rf_smote_Noisestd0.1_3features.csv', index=False)